In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import Document


text = "The quick brown fox jumps over the lazy dog"
doc = Document(
    text=text,
    metadata = {'author': "Aadi", 'category':'others'},
    id_ = '1'
)

print(doc)


Doc ID: 1
Text: The quick brown fox jumps over the lazy dog


In [ ]:
# read wikipedia page
from llama_index.readers.wikipedia import WikipediaReader

loader = WikipediaReader()
documents = loader.load_data(
    pages=['Pythagorean theorem', 'General relativity']
)

print(f"loaded {len(documents)} documents")

loaded 2 documents


In [ ]:
# manual node extracting
from llama_index.core import Document
from llama_index.core.schema import TextNode

doc = Document(text='This is text')
n1 = TextNode(text=doc.text[0:4], doc_id=doc.id_)
n2 = TextNode(text=doc.text[5:12], doc_id=doc.id_)
print(n1)
print(n2)


Node ID: 20f368f5-7966-4144-a022-c0c819d01005
Text: This
Node ID: cfd17a5e-420a-4489-990f-fe8608ebf8e2
Text: is text


In [8]:
# auto node extracting
from llama_index.core import Document
from llama_index.core.node_parser import TokenTextSplitter

doc = Document(
    text=(
    "This is sentence 1. This is sentence 2. "
    "Sentence 3 here."
    ),
    metadata={'author':'aadi'}
)

splitter = TokenTextSplitter(
    chunk_size=12,
    chunk_overlap=0,
    separator=" "
)

nodes = splitter.get_nodes_from_documents([doc])

for node in nodes:
    print(node.text)
    print(node.metadata)

Metadata length (6) is close to chunk size (12). Resulting chunks are less than 50 tokens. Consider increasing the chunk size or decreasing the size of your metadata to avoid this.
This is sentence 1.
{'author': 'aadi'}
This is sentence 2.
{'author': 'aadi'}
Sentence 3 here.
{'author': 'aadi'}


In [9]:
# Node Relationship
from llama_index.core import Document
from llama_index.core.schema import (
    TextNode,
    NodeRelationship,
    RelatedNodeInfo
)

doc = Document(text="First sentence. Second Sentence")
n1 = TextNode(text="First sentence.", node_id=doc.doc_id)
n2 = TextNode(text="Second Sentence", node_id=doc.doc_id)

n1.relationships[NodeRelationship.NEXT] = n2.node_id
n2.relationships[NodeRelationship.PREVIOUS] = n1.node_id

print(n1.relationships)
print(n2.relationships)

{<NodeRelationship.NEXT: '3'>: 'bb8b4976-e1bc-4144-8d9a-8d72f497e1af'}
{<NodeRelationship.PREVIOUS: '2'>: '720697eb-f888-44e3-b721-7b440d956193'}


In [12]:
# summary index
from llama_index.core import SummaryIndex, Document
from llama_index.core.schema import TextNode

nodes = [
    TextNode(
        text="Lionel Messi is a football player from Argentina."
    ),
    TextNode(
        text="He has won the Ballon d'Or trophy 7 times."
    ),
    TextNode(text="Lionel Messi's hometown is Rosario."),
    TextNode(text="He was born on June 24, 1987.")
]

index = SummaryIndex(nodes)

query_engine = index.as_query_engine()
response = query_engine.query("What is Messi's hometown?")
print(response)

ValueError: 
******
Could not load OpenAI model. If you intended to use OpenAI, please check your OPENAI_API_KEY.
Original error:
No API key found for OpenAI.
Please set either the OPENAI_API_KEY environment variable or openai.api_key prior to initialization.
API keys can be found or created at https://platform.openai.com/account/api-keys

To disable the LLM entirely, set llm=None.
******

In [13]:
# data index on wiki
from llama_index.core import Document, SummaryIndex
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.readers.wikipedia import WikipediaReader

loader = WikipediaReader()
documents = loader.load_data(pages=['Messi Lionel'])
parser = SimpleNodeParser.from_defaults()
nodes = parser.get_nodes_from_documents(documents)
index = SummaryIndex(nodes)

query_engine = index.as_query_engine()
while True:
    question = input("Your question: ")
    if question.lower() == "exit":
        break
    response = query_engine.query(question)
    print(response)

ValueError: 
******
Could not load OpenAI model. If you intended to use OpenAI, please check your OPENAI_API_KEY.
Original error:
No API key found for OpenAI.
Please set either the OPENAI_API_KEY environment variable or openai.api_key prior to initialization.
API keys can be found or created at https://platform.openai.com/account/api-keys

To disable the LLM entirely, set llm=None.
******

In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.core.settings import Settings

Settings.llm = OpenAI(temperature=0.8, model='gpt-3')

In [ ]:
from llama_index.llms.openai import OpenAI
llm = OpenAI(
    model="gpt-3.5-turbo-1106",
    temperature=0.2,
    max_tokens=50,
    additional_kwargs={
        "seed": 12345678,   
        "top_p": 0.5
    }
)
response = llm.complete(
    "Explain the concept of gravity in one sentence"
)
print(response)